<a href="https://colab.research.google.com/github/AmirJlr/Graph-Neural-Networks/blob/master/04-PyG/1_PyG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation :

In [ ]:
import torch; print(torch.__version__)

2.1.0+cu121


In [ ]:
import torch; print(torch.version.cuda)

12.1


In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 60.7 MB/s eta 0:00:00


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00


# PyG Introduction by Example

https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html

At its core, PyG provides the following main features:

- Data Handling of Graphs

- Common Benchmark Datasets

- Mini-batches

- Data Transforms

- Learning Methods on Graphs

- Exercises

## Data Handling of Graphs :
A graph is used to model pairwise relations (edges) between objects (nodes).

**A single graph in PyG is described by an instance of torch_geometric.data.Data**,

which holds the following attributes by default:

- **data.x**: Node feature matrix with **shape [num_nodes, num_node_features]**

- **data.edge_index**: Graph connectivity in COO format with **shape [2, num_edges]** and type torch.long

- **data.edge_attr**: Edge feature matrix with **shape [num_edges, num_edge_features]**

- **data.y**: Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]

- **data.pos**: Node position matrix with shape [num_nodes, num_dimensions]

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor(
    [[0, 1, 1, 2],
     [1, 0, 2, 1]], dtype=torch.long
)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

graph = Data(x = x, edge_index=edge_index)
graph

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
edge_index.shape

torch.Size([2, 4])

In [ ]:
### another defining :

import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

graph = Data(x=x, edge_index=edge_index.t().contiguous())
graph

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
graph.validate(raise_on_error=True)

True

Besides holding a number of **node-level**, **edge-level** or **graph-level** attributes, Data provides a number of **useful utility functions**, e.g.:

In [ ]:
graph.keys()

['x', 'edge_index']

In [ ]:
graph['x']

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [ ]:
for key, item in graph:
    print(key)
    print(item)

x
tensor([[-1.],
        [ 0.],
        [ 1.]])
edge_index
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [ ]:
graph.num_nodes

3

In [ ]:
graph.num_node_features

1

In [ ]:
graph.has_isolated_nodes()

False

In [ ]:
graph.has_self_loops()


False

In [ ]:
graph.is_directed()

False

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
graph = graph.to(device)

## Common Benchmark Datasets

Initializing a dataset is straightforward. An initialization of a dataset will automatically download its raw files and process them to the previously described Data format.

In [ ]:
# load the ENZYMES dataset (consisting of 600 graphs within 6 classes):

from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Processing...
Done!


In [ ]:
dataset

ENZYMES(600)

In [ ]:
len(dataset)

600

In [ ]:
dataset.x.shape

torch.Size([19580, 3])

In [ ]:
dataset.num_node_features

3

In [ ]:
dataset.num_classes

6

In [ ]:
data_0 = dataset[0]

In [ ]:
data_0

Data(edge_index=[2, 168], x=[37, 3], y=[1])

We can see that the first graph in the dataset contains 37 nodes, each one having 3 features. There are 168/2 = 84 undirected edges and the graph is assigned to exactly one class.

In [ ]:
data_0.keys()

['x', 'y', 'edge_index']

We can even use slices, long or bool tensors to split the dataset. E.g., to create a 90/10 train/test split :



In [ ]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [ ]:
# If you are unsure whether the dataset is already shuffled before you split,
#  you can randomly permutate it by running:

dataset = dataset.shuffle()

### Let’s download Cora, the standard benchmark dataset for semi-supervised graph node classification:

In [ ]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [ ]:
len(cora_dataset)

1

In [ ]:
cora_dataset.x.shape

torch.Size([2708, 1433])

In [ ]:
cora_dataset.num_node_features

1433

In [ ]:
cora_dataset.num_classes

7

The dataset contains only a single, undirected citation graph:

In [ ]:
cora_data = cora_dataset[0]
cora_data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
cora_data.is_undirected()

True

In [ ]:
cora_data.train_mask.sum().item()

140

In [ ]:
cora_data.val_mask.sum().item()

500

In [ ]:
cora_data.test_mask.sum().item()

1000

## Mini-batches :
Neural networks are usually trained in a batch-wise fashion.

PyG achieves parallelization over a mini-batch by **creating sparse block diagonal adjacency matrices (defined by edge_index) and concatenating feature and target matrices in the node dimension**.

This composition allows differing number of nodes and edges over examples in one batch:



### PyG contains its own torch_geometric.loader.DataLoader, which already takes care of this concatenation process. Let’s learn about it in an example:

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


**torch_geometric.data.Batch inherits from torch_geometric.data.Data** and contains an additional attribute called **batch**.

In [ ]:
for batch in loader:
    print(batch)
    print(batch.num_graphs)

DataBatch(edge_index=[2, 3990], x=[1093, 21], y=[32], batch=[1093], ptr=[33])
32
DataBatch(edge_index=[2, 3928], x=[1108, 21], y=[32], batch=[1108], ptr=[33])
32
DataBatch(edge_index=[2, 4294], x=[1075, 21], y=[32], batch=[1075], ptr=[33])
32
DataBatch(edge_index=[2, 3708], x=[940, 21], y=[32], batch=[940], ptr=[33])
32
DataBatch(edge_index=[2, 3724], x=[1028, 21], y=[32], batch=[1028], ptr=[33])
32
DataBatch(edge_index=[2, 4238], x=[1090, 21], y=[32], batch=[1090], ptr=[33])
32
DataBatch(edge_index=[2, 3686], x=[966, 21], y=[32], batch=[966], ptr=[33])
32
DataBatch(edge_index=[2, 3800], x=[986, 21], y=[32], batch=[986], ptr=[33])
32
DataBatch(edge_index=[2, 3674], x=[951, 21], y=[32], batch=[951], ptr=[33])
32
DataBatch(edge_index=[2, 3670], x=[953, 21], y=[32], batch=[953], ptr=[33])
32
DataBatch(edge_index=[2, 4280], x=[1099, 21], y=[32], batch=[1099], ptr=[33])
32
DataBatch(edge_index=[2, 4272], x=[1180, 21], y=[32], batch=[1180], ptr=[33])
32
DataBatch(edge_index=[2, 4356], x=[122

batch is a **column vector** which **maps each node to its respective graph in the batch.**

## Data Transforms:

Transforms are a common way in **torchvision** to **transform images and perform augmentation**.

 PyG comes with its own transforms, which **expect a Data object as input and return a new transformed Data object**.

 Transforms can be **chained together using torch_geometric.transforms.Compose** and are applied before saving a processed dataset on disk (pre_transform) or before accessing a graph in a dataset (transform).

In [ ]:
# Let’s look at an example,
# where we apply transforms on the ShapeNet dataset
# (containing 17,000 3D shape point clouds and per point labels from 16 shape categories):

from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

We can convert the point cloud dataset into a graph dataset by generating nearest neighbor graphs from the point clouds via transforms:



In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

dataset[0]

Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

# Learning Methods on Graphs :

After learning about data handling, datasets, loader and transforms in PyG, it’s time to **implement our first graph neural network!**

**We will use a simple GCN layer and replicate the experiments on the Cora citation dataset :**

In [ ]:
# We first need to load the Cora dataset :

from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
cora_dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
#  Now let’s implement a two-layer GCN:

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

The constructor defines two GCNConv layers which get called in the forward pass of our network.

Note that the non-linearity is not integrated in the conv calls and hence needs to be applied afterwards.

Here, we chose to use ReLU as our intermediate non-linearity and finally output a softmax distribution over the number of classes.

Let’s train this model on the training nodes for 200 epochs:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = cora_dataset[0].to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2708x1433 and 3x16)

In [ ]:
### Finally, we can evaluate our model on the test nodes:

model.eval()
pred = model(data).argmax(dim=1)

correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()

acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2708x1433 and 3x16)